In [ ]:
import pandas

In [ ]:
from wxyz.lab.widget_blueprint import T, W, Icon, Button 

# Icon
`ipywidgets` ships with an `Icon` class that will accept any of (some version) of FontAwesome icons. JupyterLab 1.x also includes the [Blueprint icons](https://blueprintjs.com/docs/#icons). There's probably an opportunity to make both work, e.g. an `IconSet`.

In [ ]:
icons = pandas.read_json("https://unpkg.com/@blueprintjs/icons@3.10.0/resources/icons/icons.json").set_index("iconName")

In [ ]:
def make_an_icon_explorer(icon=None):
    
    if icon is None:
        icon = Icon()
    out = W.Output()
    
    icon_combo = W.Select(options=sorted(icons.index), rows=1)
    group = W.Select(options=["<all>"] + sorted(set(icons["group"])), rows=1)
    display_name = W.Text(description="Display Name", disabled=True)
    T.dlink(
        (group, "value"), (icon_combo, "options"), 
        lambda group: sorted(set(icons[icons["group"] == group].index)) if group != "<all>" else sorted(icons.index)
    )
        
    with out:
        @W.interact(
            group=group,
            icon=icon_combo,
            display_name=display_name,
            size=W.IntSlider(100, min=8, max=256),
            **{c: W.IntSlider(255, max=255) for c in "red green blue".split(" ")}, 
            alpha=W.FloatSlider(1, min=0, max=1))
        def icon_explorer(**kwargs):
            display_name.value = icons.loc[kwargs["icon"]].displayName
            with icon.hold_trait_notifications():
                icon.icon = kwargs["icon"]
                icon.color = "rgba({red},{green},{blue},{alpha})".format(**kwargs)
                icon.icon_size = kwargs["size"]
    
    return out, icon

In [ ]:
if __name__ == "__main__":
    explorer, icon = make_an_icon_explorer()
    display(W.HBox([explorer, icon]))

In [ ]:
icon.color = "var(--jp-ui-font-color1)"

# Button
[Blueprint buttons](https://blueprintjs.com/docs/#core/components/button) provide a number of features.

In [ ]:
def make_a_button_explorer(btn=None):
    btn = btn or Button()
    
    icon = btn.icon if isinstance(btn.icon, Icon) else Icon(btn.icon)
    icon_right = btn.icon_right if isinstance(btn.icon_right, Icon) else Icon(btn.icon_right) 
    icon_explorer, btn.icon = make_an_icon_explorer(icon)
    icon_explorer_right, btn.icon_right = make_an_icon_explorer(icon_right)
    
    btn.icon, btn.icon_right = icon, icon_right
    
    out = W.Output()
    
    with out:
        @W.interact
        def a_button(description="A Button", minimal=False, large=False, small=False, loading=False):
            with btn.hold_trait_notifications():
                btn.description = description
                btn.minimal = minimal
                btn.large = large
                btn.small = small
                btn.loading = loading
    
    return btn, icon_explorer, out, icon_explorer_right

In [ ]:
if __name__ == "__main__":
    btn_widgets = make_a_button_explorer()
    display(
        W.VBox([
            W.HBox(btn_widgets[1:]),
            btn_widgets[0],
        ])
    )